# SarXarray Demo Notebook

In this Jupyter Notebook, we demonstrate the following operations using `sarxarray`:

- Load an SLC stack in binary format into a `xarray.Dataset` object;
- Append variable data fields;
- Create an MRM of a subset of the SLC stack;
- Apply point selection to a subset of the SLC stack;
- Export the selection results in Zarr format

The data needed for this Notebook is sored in the following pth on Spider: `/project/caroline/Public/demo_sarxarray`.

In [ ]:
# To execute this notebook, you may need to install matplotlib
# For example, execute this block to install matplotlib into "sarxarray" environment
# ! conda install -n sarxarray -c conda-forge matplotlib -y

In [ ]:
import numpy as np
from pathlib import Path
import sarxarray

### Load a SLC stack

In [ ]:
# You many need to change this cell to your local data directory
path = Path('../../data/nl_veenweiden_s1_asc_t088/')

In [ ]:
# Make a list of SLCs to read
f_slc = 'slc_srd_nl_veenweiden.raw' 
list_slcs = [p/f_slc for p in path.rglob("????????")]
list_slcs

In [ ]:
# Metadata of the stack
shape=(9914, 41174)
dtype = np.dtype([('re', np.float32), ('im', np.float32)])

In [ ]:
stack = sarxarray.from_binary(list_slcs, shape, dtype=dtype)
stack

In [ ]:
stack.complex

In [ ]:
# Can retruck, but this will reduce the performance, since the mmap reading on binary data has a fixed trunk
stack.chunk({"azimuth":1000, "range":1000, "time":1 })

In [ ]:
stack.complex

### Append a new data field: `h2ph`

In [ ]:
# Make a list of h2ph files to read
f_h2ph = 'h2ph_nl_veenweiden.raw' 
list_h2phs = [p/f_h2ph for p in path.rglob("????????")]
list_h2phs

In [ ]:
h2phs = sarxarray.from_binary(list_h2phs, shape, vlabel="h2ph", dtype=np.float32)
h2phs

In [ ]:
stack = stack.merge(h2phs)
stack

### Create MRM of a subset

In [ ]:
mrm = stack.slcstack.mrm()
mrm

In [ ]:
# Compute a subset of the mrm
mrm_subset = mrm[5000:5500, 10000:10500]
mrm_subset = mrm_subset.compute()

In [ ]:
# Visualize
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
ax.imshow(mrm_subset)
mrm_subset.plot(robust=True, ax=ax)

### Export to a Space Time Matrix

In [ ]:
stack_subset = stack.sel(azimuth=range(6000,6500), range=range(11000,11500))

In [ ]:
stm = stack_subset.slcstack.point_selection(threshold=2, method="amplitude_dispersion")
stm

In [ ]:
# Export to Zarr
stm.to_zarr("stm.zarr")

In [ ]:
# Load exported STM in Zarr
import xarray as xr
stm_read = xr.open_zarr("stm.zarr")
stm_read